# HW1: Logistic Regression

This class is about models and algorithms for discrete data. This homework will have all 3 ingredients:
* **Data**: the results from all college football games in the 2023 season
* **Model**: The *Bradely-Terry* model for predicting the winners of football game. The Bradley-Terry model is just logistic regression.
* **Algorithm**: We will implement two ways of fitting logistic regression: gradient descent and Newton's method

## The Bradley-Terry Model

In the Bradley-Terry Model, we give team $k$ a team-effect $\beta_k$. Basically, higher $\beta_k$ (relatively speaking), means that team $k$ is a better team.
The Bradley-Terry model formalizes this intution by modeling the log odds of team $k$ beating team $k'$ by the difference in their team effects, $\beta_k - \beta_{k'}$.

Let $i = 1,\ldots, n$ index games, and let $h(i) \in \{1,\ldots,K\}$ and $a(i) \in \{1,\ldots,K\}$ denote the indices of the home and away teams, respectively.
Let $Y_i \in \{0,1\}$ denote whether the home team won.
Under the Bradley-Terry model,
\begin{equation*}
  Y_i \sim \mathrm{Bern}\big(\sigma(\beta_{h(i)} - \beta_{a(i)}) \big),
\end{equation*}
where $\sigma(\cdot)$ is the sigmoid function. We can view this model as a logistic regression model with covariates $x_i \in \mathbb{R}^K$ where,
\begin{align*}
x_{i,k} &=
\begin{cases}
+1 &\text{if } h(i) = k \\
-1 &\text{if } a(i) = k \\
0 &\text{o.w.},
\end{cases}
\end{align*}
and parameters $\beta \in \mathbb{R}^K$.

## Data

We use the results of college football games in the fall 2023 season, which are available from the course github page and loaded for you below.

The data comes as a list of the outcomes of individual games. You'll need to wrangle the data to get it into a format that you can feed into the Bradley-Terry model.

In [38]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

In [203]:
allgames = pd.read_csv("https://raw.githubusercontent.com/slinderman/stats305b/winter2025/data/01_allgames.csv")

## Problem 0: Preprocessing

Preprocess the data to drop games with nan scores, construct the covariate matrix $X$, construct the response vector $y$, and do any other preprocessing you find useful.

In [204]:
# drop games with nan scores
# store the results in a new dataframe called df
df = allgames.dropna(subset=['Home Points', 'Away Points'])
# to avoid bias, drop all the games where the points are tied.
df = df[df["Away Points"] != df["Home Points"]]

# in Bradley-Terry model, constructing the response vector y
# the covariate matrix X is very sparse. we can one column for each team,
# and in each row, value equals 1 if the team is the home team, and equals -1 if the team is the away team,
# and equals 0 for all other teams.
# therefore, there is only one possible covariate matrix.
points_difference = df["Home Points"] - df["Away Points"]
y = (points_difference > 0)*1 # consider the home team won if the home team had higher points in the game
teams = sorted(set(df["Away Team"]).union(set(df["Home Team"])))
X_df = pd.DataFrame(0, index=df.index, columns=teams)
for i, game in df.iterrows():
  home_team = game["Home Team"]
  away_team = game["Away Team"]
  X_df.loc[i, home_team] = 1
  X_df.loc[i, away_team] = -1
# convert into tensors
#type(y), type(X_df)
y = torch.tensor(y.values, dtype=torch.float32).unsqueeze(1)
X = torch.tensor(X_df.values, dtype=torch.float32)
y.shape, X.shape

(torch.Size([3574, 1]), torch.Size([3574, 703]))

## Problem 1: Loss function

Write a function to compute the loss, $L(\beta)$ defined be

\begin{equation*}
  L(\beta) = -\frac{1}{n} \sum_{i=1}^n \log p(y_i \mid x_i; \beta) + \frac{\gamma}{2} \| \beta \|_2^2
\end{equation*}
where $\gamma$ is a hyperparameter that controls the strength of your $\ell_2$ regularization.

You may want to use the `torch.distributions.Bernoulli` class.

In [205]:
# your code here
def compute_loss(beta, X, y, gamma):
  average_log_likelihood = torch.distributions.Bernoulli(probs = torch.sigmoid(X @ beta)).log_prob(y).mean()
  loss = -1 * average_log_likelihood + gamma / 2 * (torch.norm(beta, 2) ** 2) # add l2 regularization
  return loss

In [206]:
# unit testing
torch.manual_seed(0)
beta = torch.rand([X.shape[1],1], dtype=torch.float32, requires_grad=True)
gamma = 0
compute_loss(beta, X, y, gamma)

tensor(0.7100, grad_fn=<AddBackward0>)

In [207]:
compute_loss(beta, X, y, gamma).shape

torch.Size([])

## Problem 2: Gradient Descent

### Problem 2.1 Implementing and checking your gradients


Write a function to compute the gradient of the average negative log likelihood and check your output against the results obtained by PyTorch's automatic differentiation functionality.

The gradient is computed as:
$$
\nabla L(\beta) = -\frac{1}{n} \sum_{i=1}^{n} \left[(y_i - \sigma(\beta^\intercal x_i)) x_i\right] + \gamma \beta
$$

Note that this is a vector with the same shape as $\beta$.

In [208]:
# your code here
def compute_gradient(beta, X, y, gamma):
  grad_average_log_likelihood = -1/(y.shape[0]) * (X.T @ (y - torch.sigmoid(X @ beta)))
  grad_regularization = (gamma * beta)
  gradient = grad_average_log_likelihood + grad_regularization
  return gradient

In [209]:
loss = compute_loss(beta, X, y, gamma)
loss.backward()
auto_gradient = beta.grad

In [210]:
gradient = compute_gradient(beta, X, y, gamma)

In [211]:
torch.allclose(gradient, auto_gradient)

True

### Problem 2.2: Implement Gradient Descent


Now, use gradient descent to fit your Bradley-Terry model to the provided data.

Deliverables for this question:
1. Code the implements gradient descent to fit your Bradley-Terry model to the provided data.
2. A plot of the loss curve of your algorithm and a brief discussion if it makes sense or not
3. A plot of the histogram of the fitted values of $\beta$
4. The top 10 teams from your ranking, and a discussion of whether this ranking makes sense or not.

In [ ]:
# your code here (you can use multiple code and markdown cells to organize your answer)

In [233]:
# gradient descent

# parameters
torch.manual_seed(0)
learning_rate = 0.5
gamma = 0.2
# initial beta
beta = torch.rand([X.shape[1],1], dtype=torch.float32, requires_grad=True)

def gradient_descent_step(beta, learning_rate):
  #compute gradient
  current_gradient = compute_gradient(beta, X, y, gamma)
  next_beta = beta - learning_rate * current_gradient
  return next_beta

num_iterations = 100
list_of_loss = [compute_loss(beta, X, y, gamma)]
for _ in range(num_iterations):
  beta = gradient_descent_step(beta, learning_rate)
  list_of_loss.append(compute_loss(beta, X, y, gamma))

In [234]:
loss_tensor = torch.as_tensor(list_of_loss)
loss_tensor

tensor([24.1480, 19.6877, 16.0756, 13.1503, 10.7812,  8.8627,  7.3090,  6.0507,
         5.0318,  4.2065,  3.5382,  2.9970,  2.5587,  2.2038,  1.9163,  1.6836,
         1.4950,  1.3424,  1.2187,  1.1186,  1.0375,  0.9718,  0.9187,  0.8756,
         0.8407,  0.8125,  0.7896,  0.7711,  0.7561,  0.7439,  0.7341,  0.7261,
         0.7197,  0.7144,  0.7102,  0.7068,  0.7040,  0.7018,  0.6999,  0.6985,
         0.6973,  0.6963,  0.6955,  0.6949,  0.6944,  0.6940,  0.6936,  0.6933,
         0.6931,  0.6929,  0.6928,  0.6927,  0.6926,  0.6925,  0.6925,  0.6924,
         0.6924,  0.6923,  0.6923,  0.6923,  0.6923,  0.6922,  0.6922,  0.6922,
         0.6922,  0.6922,  0.6922,  0.6922,  0.6922,  0.6922,  0.6922,  0.6922,
         0.6922,  0.6922,  0.6922,  0.6922,  0.6922,  0.6922,  0.6922,  0.6922,
         0.6922,  0.6922,  0.6922,  0.6922,  0.6922,  0.6922,  0.6922,  0.6922,
         0.6922,  0.6922,  0.6922,  0.6922,  0.6922,  0.6922,  0.6922,  0.6922,
         0.6922,  0.6922,  0.6922,  0.69

## Problem 3: Newton's Method

Now, use Newton's method to fit your Bradley-Terry model to the provided data.


### Problem 3.1 The Hessian



#### Problem 3.1.1. Implement and check the Hessian
Write a function to compute the Hessian of the average negative log likelihood and check your answer against the output of `from torch.autograd.functional.hessian`.

#### Problem 3.1.2: Positive definiteness

Compute the Hessian at the point $\beta = 0$ without regularization (set $\gamma = 0$). Unless you've done sort of pre-processing, it's probably singular.

In [ ]:
# your code here

#### Problem 3.1.3

Describe intuitively and mathematically what it means for the Hessian of the negative log likelihood to be singular in the context of this data and model

*your answer here*

#### Problem 3.1.4

Give a hypothesis for why the Hessian in this dataset and model is singular, and provide empirical evidence to support your hypothesis.

*your answer here*

In [ ]:
# your code here

#### Problem 3.1.5

Explain why the Hessian is invertible when $\gamma > 0$.

*your answer here*

### Problem 3.2: Implement Newton's method

Now, use Newton's method to fit your $\ell_2$-regularized Bradley-Terry model to the provided data.

Deliverables for this question:
1. Code the implements Newton's method to fit your Bradley-Terry model to the provided data.
2. A plot of the loss curves from Newton's method and from gradient descent, using the same regularization strength $\gamma$ and initialization $\beta_0$. Briefly discuss the results and compare their rates of convergence.
3. A plot of the histogram of the fitted values of $\beta$
4. The top 10 teams from your ranking, and a discussion of whether this ranking makes sense or not.

In [ ]:
# your code here (you can use multiple code and markdown cells to organize your answer)

## Problem 4: Model criticism and revision

Let's take another look the Bradley-Terry model from earlier and think about improvements we can make.


### Problem 4.1: Improvements to Bradley-Terry Model
Choose one way to improve the Bradley-Terry model. Discuss *a priori* why you think this change will improve the model and implement your change.

*your answer here*

### Problem 4.2: Evaluation
Assess whether or not your change was an improvement or not. Provide empirical evidence by evaluating performance on a held out test set and include at least one plot supporting your assessment.

*your answer here*

### Problem 4.3: Reflection
Reflecting on the analysis we've conducted in this assignemnt, which conference is best? Is there a significant difference? Please justify your answer.

*your answer here*

## Submission Instructions

**Formatting:** check that your code does not exceed 80 characters in line width. You can set _Tools &rarr; Settings &rarr; Editor &rarr; Vertical ruler column_ to 80 to see when you've exceeded the limit.

**Converting to PDF** The simplest way to convert to PDF is to use the "Print to PDF" option in your browser. Just make sure that your code and plots aren't cut off, as it may not wrap lines.

**Alternatively** You can download your notebook in .ipynb format and use the following commands to convert it to PDF.  Then run the following command to convert to a PDF:
```
jupyter nbconvert --to pdf <yourlastname>_hw<number>.ipynb
```
(Note that for the above code to work, you need to rename your file `<yourlastname>_hw<number>.ipynb`)

**Installing nbconvert:**

If you're using Anaconda for package management,
```
conda install -c anaconda nbconvert
```

**Upload** your .pdf file to Gradescope. Please tag your questions correctly! I.e., for each question, all of and only the relevant sections are tagged.

Please post on Ed or come to OH if there are any other problems submitting the HW.